# Task
Develop a time series forecasting model using the candlestick data from `all_samples_numeric.pkl`, train the model, predict future sequences, and visualize the predictions by reconstructing candlestick images for comparison between actual and predicted future sequences.

## Load Dataset

### Subtask:
Load the `all_samples_numeric.pkl` dataset using the pickle library. Each sample is a dictionary containing 'past' and 'future' candlestick sequences.


**Reasoning**:
The subtask requires loading a pickle file. This code block will import the pickle library, open the specified file, and load its content into a variable named `data`.



In [5]:
import pickle

with open('/content/all_samples_numeric.pkl', 'rb') as f:
    data = pickle.load(f)

print(f"Dataset loaded successfully. It contains {len(data)} samples.")
print(f"Type of loaded data: {type(data)}")
if isinstance(data, list) and len(data) > 0:
    print(f"Type of first sample: {type(data[0])}")
    print(f"Keys in first sample: {list(data[0].keys())}")

Dataset loaded successfully. It contains 2158 samples.
Type of loaded data: <class 'list'>
Type of first sample: <class 'dict'>
Keys in first sample: ['past', 'future', 'input_size', 'output_size']


## Split Dataset

### Subtask:
Split the loaded dataset into training, validation, and test sets. Ensure the split maintains the structure of the samples (past and future sequences).


**Reasoning**:
The subtask requires splitting the dataset into training, validation, and test sets. This code block will import the necessary function from scikit-learn and perform the data splitting according to the specified ratios.



In [6]:
from sklearn.model_selection import train_test_split

# Split into training (70%) and temporary (30%) sets
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)

# Split the temporary set into validation (15%) and test (15%) sets
# Since temp_data is 30% of the total, we split it 50/50 for 15% each
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"Total samples: {len(data)}")
print(f"Training samples: {len(train_data)} ({len(train_data)/len(data):.2%})")
print(f"Validation samples: {len(val_data)} ({len(val_data)/len(data):.2%})")
print(f"Test samples: {len(test_data)} ({len(test_data)/len(data):.2%})")

Total samples: 2158
Training samples: 1510 (69.97%)
Validation samples: 324 (15.01%)
Test samples: 324 (15.01%)


## Define PyTorch Dataset and DataLoader

### Subtask:
Create a custom PyTorch Dataset to handle the candlestick sequences. Implement a custom collate_fn for the DataLoader to handle variable sequence lengths by padding the sequences appropriately for batch processing.


**Reasoning**:
The subtask requires defining a custom PyTorch Dataset and a custom collate_fn for handling variable-length sequences. This code block will import the necessary PyTorch modules, define the `CandlestickDataset` class to prepare individual samples, and implement the `collate_fn` to pad sequences for batch processing.



In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class CandlestickDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        past_sequence = torch.tensor(sample['past'], dtype=torch.float32)
        future_sequence = torch.tensor(sample['future'], dtype=torch.float32)
        return {
            'past': past_sequence,
            'future': future_sequence,
            'past_len': len(past_sequence),
            'future_len': len(future_sequence)
        }

def collate_fn(batch):
    past_sequences = [item['past'] for item in batch]
    future_sequences = [item['future'] for item in batch]
    past_lengths = torch.tensor([item['past_len'] for item in batch], dtype=torch.long)
    future_lengths = torch.tensor([item['future_len'] for item in batch], dtype=torch.long)

    # Pad sequences to the maximum length in the batch
    # batch_first=True makes the output tensor (batch_size, seq_len, feature_dim)
    padded_past = pad_sequence(past_sequences, batch_first=True, padding_value=0.0)
    padded_future = pad_sequence(future_sequences, batch_first=True, padding_value=0.0)

    return {
        'past': padded_past,
        'past_len': past_lengths,
        'future': padded_future,
        'future_len': future_lengths
    }

print("CandlestickDataset class and collate_fn function defined successfully.")

CandlestickDataset class and collate_fn function defined successfully.


**Reasoning**:
Now that the `CandlestickDataset` and `collate_fn` are defined, I will instantiate datasets and dataloaders for the training, validation, and test sets as per the instructions. I will use a `batch_size` of 32.



In [8]:
batch_size = 32

# Instantiate datasets
train_dataset = CandlestickDataset(train_data)
val_dataset = CandlestickDataset(val_data)
test_dataset = CandlestickDataset(test_data)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

print(f"Train DataLoader created with {len(train_dataloader)} batches.")
print(f"Validation DataLoader created with {len(val_dataloader)} batches.")
print(f"Test DataLoader created with {len(test_dataloader)} batches.")

# Verify a batch
for batch in train_dataloader:
    print(f"Sample batch 'past' shape: {batch['past'].shape}")
    print(f"Sample batch 'past_len' shape: {batch['past_len'].shape}")
    print(f"Sample batch 'future' shape: {batch['future'].shape}")
    print(f"Sample batch 'future_len' shape: {batch['future_len'].shape}")
    break

Train DataLoader created with 48 batches.
Validation DataLoader created with 11 batches.
Test DataLoader created with 11 batches.
Sample batch 'past' shape: torch.Size([32, 135, 4])
Sample batch 'past_len' shape: torch.Size([32])
Sample batch 'future' shape: torch.Size([32, 44, 4])
Sample batch 'future_len' shape: torch.Size([32])


## Build RNN/LSTM Model

### Subtask:
Define a PyTorch RNN or LSTM model architecture. The model should accept a 'past' sequence of shape [seq_len, 4] as input and output a 'future' sequence of shape [future_len, 4].


**Reasoning**:
The subtask requires defining a PyTorch LSTM model. This code block will import the necessary PyTorch modules, define the `CandlestickPredictor` class with LSTM layers, and then instantiate and print the model to verify its architecture.



In [9]:
import torch.nn as nn

class CandlestickPredictor(nn.Module):
    def __init__(self, input_size=4, output_size=4, hidden_size=128, num_layers=2):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size # Store output_size to use in forward method

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x, lengths):
        # Pack padded sequence
        packed_input = nn.utils.rnn.pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)

        # Pass through LSTM
        packed_output, _ = self.lstm(packed_input)

        # Unpack padded sequence
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)

        # Apply linear layer
        predictions = self.linear(output)

        # The predictions will have the shape (batch_size, seq_len, output_size)
        # We need to ensure that we output 'future_len' steps, so we take the full sequence if it's longer
        # or pad with zeros if it's shorter than the max future_len in a batch. For training, we predict for
        # all output timesteps provided by the LSTM output.
        return predictions

# Instantiate the model
# Assuming input_size and output_size are 4 for (open, high, low, close)
model = CandlestickPredictor(input_size=4, output_size=4, hidden_size=128, num_layers=2)

print(model)
print("CandlestickPredictor model defined and instantiated successfully.")

CandlestickPredictor(
  (lstm): LSTM(4, 128, num_layers=2, batch_first=True)
  (linear): Linear(in_features=128, out_features=4, bias=True)
)
CandlestickPredictor model defined and instantiated successfully.


## Build RNN/LSTM Model (Retry 1)

### Subtask:
Modify the PyTorch RNN or LSTM model architecture (`CandlestickPredictor`) to explicitly output a 'future' sequence of `future_len` steps, as required, using an encoder-decoder structure.


**Reasoning**:
The subtask requires modifying the `CandlestickPredictor` model to an encoder-decoder architecture to explicitly output a future sequence of `future_len` steps. This code block will update the `__init__` and `forward` methods as per the instructions to implement this architecture.



In [10]:
import torch.nn as nn

class CandlestickPredictor(nn.Module):
    def __init__(self, input_size=4, output_size=4, hidden_size=128, num_layers=2):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size

        # Encoder LSTM
        self.encoder_lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Decoder LSTM - takes previous prediction (output_size) as input
        self.decoder_lstm = nn.LSTM(output_size, hidden_size, num_layers, batch_first=True)
        # Linear layer to map decoder's hidden state to output_size features
        self.decoder_linear = nn.Linear(hidden_size, output_size)

    def forward(self, x, past_lengths, future_lengths):
        batch_size = x.size(0)

        # Encoder
        # Pack padded sequence
        packed_input = nn.utils.rnn.pack_padded_sequence(x, past_lengths.cpu(), batch_first=True, enforce_sorted=False)

        # Pass through encoder LSTM. We only need the final hidden and cell states.
        _, (hidden_state, cell_state) = self.encoder_lstm(packed_input)

        # Decoder
        # Determine the maximum future_len in the current batch
        max_future_len_in_batch = torch.max(future_lengths).item()

        # Create an initial 'start token' input for the decoder (e.g., a tensor of zeros)
        # This should have shape (batch_size, 1, self.output_size)
        decoder_input = torch.zeros(batch_size, 1, self.output_size, device=x.device)

        decoder_outputs = []

        # Autoregressive decoding loop
        for _ in range(max_future_len_in_batch):
            # Pass through decoder LSTM
            output, (hidden_state, cell_state) = self.decoder_lstm(decoder_input, (hidden_state, cell_state))

            # Apply linear layer to get prediction for current timestep
            prediction = self.decoder_linear(output)

            # Store the prediction
            decoder_outputs.append(prediction)

            # Use the prediction as the input for the next timestep (detach to prevent backprop through past unrolling)
            decoder_input = prediction.detach()

        # Concatenate the predictions along the sequence dimension
        predictions = torch.cat(decoder_outputs, dim=1)

        return predictions

# Instantiate the modified model
model = CandlestickPredictor(input_size=4, output_size=4, hidden_size=128, num_layers=2)

print(model)
print("CandlestickPredictor model modified to encoder-decoder architecture.")

CandlestickPredictor(
  (encoder_lstm): LSTM(4, 128, num_layers=2, batch_first=True)
  (decoder_lstm): LSTM(4, 128, num_layers=2, batch_first=True)
  (decoder_linear): Linear(in_features=128, out_features=4, bias=True)
)
CandlestickPredictor model modified to encoder-decoder architecture.


## Train Model

### Subtask:
Implement the training loop for the PyTorch model. Use `MSELoss` as the criterion for regression. Train the model over multiple epochs, using the DataLoader for batching. Track and report the training and validation loss per epoch.


**Reasoning**:
The subtask requires implementing a full training loop with training and validation steps, including device setup, loss function, optimizer, and proper handling of padded sequences for loss calculation. This code block will fulfill all these requirements.



In [18]:
import torch.optim as optim

# 1. Set up the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

if torch.cuda.is_available():
    print("CUDA is available! Training on GPU.")
else:
    print("CUDA is not available. Training on CPU.")

# 2. Define the loss function
criterion = nn.MSELoss(reduction='sum') # Using sum reduction to manually average later

# 3. Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Define the number of training epochs
num_epochs = 1000

print(f"Training on device: {device}")

# Lists to store epoch-wise losses
train_losses = []
val_losses = []

# 5. Implement the training loop
for epoch in range(num_epochs):
    model.train() # Set the model to training mode
    total_train_loss = 0
    total_train_samples = 0

    for batch_idx, batch in enumerate(train_dataloader):
        # 5.d. Move batch data to the appropriate device
        past_sequences = batch['past'].to(device)
        past_lengths = batch['past_len'].to(device)
        future_sequences = batch['future'].to(device)
        future_lengths = batch['future_len'].to(device)

        optimizer.zero_grad() # 5.g. Zero the gradients

        # 5.e. Perform a forward pass
        predictions = model(past_sequences, past_lengths, future_lengths)

        # 5.f. Calculate the loss, considering actual sequence lengths
        loss = 0
        for i in range(predictions.size(0)): # Iterate over batch_size
            actual_future_len = future_lengths[i]
            # Only consider the actual future_len for loss calculation
            loss += criterion(predictions[i, :actual_future_len, :], future_sequences[i, :actual_future_len, :])

        loss.backward() # 5.g. Compute gradients
        optimizer.step() # 5.g. Update weights

        total_train_loss += loss.item()
        total_train_samples += past_sequences.size(0) # Number of samples in the batch

    avg_train_loss = total_train_loss / total_train_samples
    train_losses.append(avg_train_loss)

    # 6. Implement the validation step
    model.eval() # Set the model to evaluation mode
    total_val_loss = 0
    total_val_samples = 0

    with torch.no_grad(): # 6.b. Disable gradient calculations
        for batch_idx, batch in enumerate(val_dataloader):
            # 6.d. Move batch data to the appropriate device
            past_sequences = batch['past'].to(device)
            past_lengths = batch['past_len'].to(device)
            future_sequences = batch['future'].to(device)
            future_lengths = batch['future_len'].to(device)

            # 6.e. Perform a forward pass
            predictions = model(past_sequences, past_lengths, future_lengths)

            # 6.f. Calculate the loss, considering actual sequence lengths
            loss = 0
            for i in range(predictions.size(0)): # Iterate over batch_size
                actual_future_len = future_lengths[i]
                # Only consider the actual future_len for loss calculation
                loss += criterion(predictions[i, :actual_future_len, :], future_sequences[i, :actual_future_len, :])

            total_val_loss += loss.item()
            total_val_samples += past_sequences.size(0)

    avg_val_loss = total_val_loss / total_val_samples
    val_losses.append(avg_val_loss)

    # 7. Print the average training and validation loss for each epoch
    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

print("Training complete.")

CUDA is available! Training on GPU.
Training on device: cuda
Epoch 1/1000 | Train Loss: 2.5632 | Val Loss: 2.6301
Epoch 2/1000 | Train Loss: 2.4928 | Val Loss: 2.8844
Epoch 3/1000 | Train Loss: 2.4817 | Val Loss: 2.4622
Epoch 4/1000 | Train Loss: 2.3626 | Val Loss: 2.6362
Epoch 5/1000 | Train Loss: 2.3100 | Val Loss: 2.4205
Epoch 6/1000 | Train Loss: 2.3206 | Val Loss: 2.5078
Epoch 7/1000 | Train Loss: 2.3128 | Val Loss: 2.3909
Epoch 8/1000 | Train Loss: 2.3341 | Val Loss: 2.7433
Epoch 9/1000 | Train Loss: 2.4135 | Val Loss: 2.5136
Epoch 10/1000 | Train Loss: 2.1975 | Val Loss: 2.6170
Epoch 11/1000 | Train Loss: 2.2555 | Val Loss: 2.4174
Epoch 12/1000 | Train Loss: 2.1360 | Val Loss: 2.3440
Epoch 13/1000 | Train Loss: 2.0976 | Val Loss: 2.3520
Epoch 14/1000 | Train Loss: 2.1171 | Val Loss: 2.3342
Epoch 15/1000 | Train Loss: 2.0290 | Val Loss: 2.3091
Epoch 16/1000 | Train Loss: 1.9952 | Val Loss: 2.4314
Epoch 17/1000 | Train Loss: 1.9953 | Val Loss: 2.2600
Epoch 18/1000 | Train Loss: 2.

## Predict on Test Data

### Subtask:
After training, use the trained model to predict future candlestick sequences for a selection of samples from the test set.


**Reasoning**:
The subtask requires using the trained model to predict future candlestick sequences on a selection of samples from the test set. This code block will set the model to evaluation mode, iterate through a few batches of the test data, make predictions, and store the actual past, actual future, and predicted future sequences for later visualization.



In [19]:
model.eval() # Set the model to evaluation mode

# Initialize lists to store actual and predicted sequences
actual_past_sequences = []
actual_future_sequences = []
predicted_future_sequences = []

num_samples_to_predict = 20 # Predict for a selection of samples for visualization

with torch.no_grad(): # Disable gradient calculations
    sample_count = 0
    for batch in test_dataloader:
        if sample_count >= num_samples_to_predict: # Stop after collecting enough samples
            break

        # Move batch data to the appropriate device
        past_sequences_batch = batch['past'].to(device)
        past_lengths_batch = batch['past_len'].to(device)
        future_sequences_batch = batch['future'].to(device)
        future_lengths_batch = batch['future_len'].to(device)

        # Make predictions
        predictions_batch = model(past_sequences_batch, past_lengths_batch, future_lengths_batch)

        # Store sequences
        for i in range(past_sequences_batch.size(0)): # Iterate over individual samples in the batch
            if sample_count >= num_samples_to_predict:
                break

            # Retrieve original (unpadded) sequences based on their lengths
            actual_past = past_sequences_batch[i, :past_lengths_batch[i]].cpu().numpy()
            actual_future = future_sequences_batch[i, :future_lengths_batch[i]].cpu().numpy()

            # Predicted future might be longer than actual future, or padded, so we trim/align it
            # The predictions tensor has shape (batch_size, max_future_len_in_batch, output_size)
            # We need to take up to the actual future length for comparison.
            predicted_future = predictions_batch[i, :future_lengths_batch[i]].cpu().numpy()

            actual_past_sequences.append(actual_past)
            actual_future_sequences.append(actual_future)
            predicted_future_sequences.append(predicted_future)
            sample_count += 1

print(f"Generated predictions for {len(actual_past_sequences)} samples from the test set.")
print(f"Length of first actual_past_sequence: {len(actual_past_sequences[0])}")
print(f"Length of first actual_future_sequence: {len(actual_future_sequences[0])}")
print(f"Length of first predicted_future_sequence: {len(predicted_future_sequences[0])}")


Generated predictions for 20 samples from the test set.
Length of first actual_past_sequence: 131
Length of first actual_future_sequence: 43
Length of first predicted_future_sequence: 43


## Visualize Predictions

### Subtask:
For each selected test sample, generate a visual plot using Matplotlib. The plot should show a side-by-side comparison: on the left, the input 'past' sequence concatenated with the 'actual future' sequence; on the right, the input 'past' sequence concatenated with the 'predicted future' sequence. Include appropriate labels and legends for clarity.

**Reasoning**:
The subtask requires visualizing the model's predictions by plotting reconstructed candlestick images. This code block will import Matplotlib, iterate through a selected number of test samples, reconstruct images for the past, actual future, and predicted future sequences, and then display them side-by-side for comparison.

In [1]:
import matplotlib.pyplot as plt

def plot_candlestick_prediction(past_sequence, actual_future_sequence, predicted_future_sequence, sample_idx):
    # Concatenate past and future for actual and predicted views
    combined_actual = np.concatenate((past_sequence, actual_future_sequence), axis=0)
    combined_predicted = np.concatenate((past_sequence, predicted_future_sequence), axis=0)

    # Reconstruct images
    img_width = 300 # Define a fixed width for consistent visualization
    img_height = 100 # Define a fixed height

    # Adjust image width based on total sequence length
    total_len_actual = len(combined_actual)
    total_len_predicted = len(combined_predicted)
    # Choose the larger sequence length to determine the visual width ratio for images
    max_total_len = max(total_len_actual, total_len_predicted)

    # Calculate an appropriate image width based on the number of candles
    # Ensure min_candle_width is at least 1 pixel per candle
    min_candle_width = 3
    actual_img_width = max(img_width, max_total_len * min_candle_width)
    predicted_img_width = max(img_width, max_total_len * min_candle_width)

    img_actual = reconstruct_candlestick_image(combined_actual, img_width=actual_img_width, img_height=img_height)
    img_predicted = reconstruct_candlestick_image(combined_predicted, img_width=predicted_img_width, img_height=img_height)

    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle(f'Sample {sample_idx+1}: Candlestick Prediction Comparison', fontsize=16)

    # Plot Actual vs. Predicted
    axes[0].imshow(img_actual, cmap='gray', origin='upper') # origin='upper' because 0 is at top in our reconstruction
    axes[0].set_title('Past + Actual Future')
    axes[0].axis('off')

    axes[1].imshow(img_predicted, cmap='gray', origin='upper')
    axes[1].set_title('Past + Predicted Future')
    axes[1].axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# Visualize a few samples
num_visualizations = min(5, len(actual_past_sequences)) # Visualize up to 5 samples

print(f"Generating visualizations for {num_visualizations} samples...")

for i in range(num_visualizations):
    plot_candlestick_prediction(
        actual_past_sequences[i],
        actual_future_sequences[i],
        predicted_future_sequences[i],
        i
    )

print("Visualizations complete.")

NameError: name 'actual_past_sequences' is not defined

## Reconstruct Images from Candlestick Data

### Subtask:
Develop a function to reconstruct candlestick images from the normalized numeric data. This function should take a sequence (past, true future, or predicted future) and convert it into an image representation. Ensure the y-axis is flipped correctly (0 at the top, 1 at the bottom) as specified.

**Reasoning**:
The subtask requires a function to reconstruct candlestick images. This code block will define `reconstruct_candlestick_image`, which takes normalized OHLC data and converts it into a grayscale image based on the candlestick properties (open, high, low, close). It also handles the specified y-axis flipping.

In [13]:
from PIL import Image
import numpy as np

def reconstruct_candlestick_image(ohlc_sequence, img_width=100, img_height=100):
    """
    Reconstructs a grayscale candlestick image from a normalized OHLC sequence.
    The image height corresponds to the 0-1 normalized price range.
    The y-axis is flipped so that 0 is at the top and 1 is at the bottom.

    Args:
        ohlc_sequence (np.ndarray): A 2D NumPy array of shape (seq_len, 4)
                                   where each row is [open, high, low, close],
                                   normalized between 0 and 1.
        img_width (int): The desired width of the output image.
        img_height (int): The desired height of the output image (representing price range).

    Returns:
        PIL.Image.Image: A grayscale PIL Image representing the candlestick pattern.
    """
    if len(ohlc_sequence) == 0:
        return Image.new('L', (img_width, img_height), color=0) # Return black image for empty sequence

    image_data = np.zeros((img_height, img_width), dtype=np.uint8) # Grayscale image

    num_candles = ohlc_sequence.shape[0]
    candle_width_pixels = max(1, img_width // num_candles)
    # Ensure we don't go out of bounds if candle_width_pixels * num_candles < img_width

    for i in range(num_candles):
        open_price, high_price, low_price, close_price = ohlc_sequence[i]

        # Convert normalized prices (0-1) to pixel coordinates (0 to img_height-1)
        # Y-axis is flipped: 0 is top, img_height-1 is bottom
        open_px = int((1 - open_price) * (img_height - 1))
        high_px = int((1 - high_price) * (img_height - 1))
        low_px = int((1 - low_price) * (img_height - 1))
        close_px = int((1 - close_price) * (img_height - 1))

        # Determine candle color based on open vs close
        if close_price >= open_price:
            # Green candle (price went up or stayed same) - represented as lighter gray
            candle_color = 200 # Light gray
        else:
            # Red candle (price went down) - represented as darker gray
            candle_color = 100 # Dark gray

        # Calculate x-coordinates for the current candle
        x_start = i * candle_width_pixels
        x_end = min(img_width, (i + 1) * candle_width_pixels) - 1

        # Draw the body of the candle (between open and close)
        body_top = min(open_px, close_px)
        body_bottom = max(open_px, close_px)
        image_data[body_top:body_bottom + 1, x_start:x_end + 1] = candle_color

        # Draw the wick (between high and low)
        wick_top = high_px
        wick_bottom = low_px
        # Draw the wick as a thin line in the center of the candle's pixel width
        wick_center_x = (x_start + x_end) // 2
        if x_start <= wick_center_x <= x_end:
            image_data[wick_top:wick_bottom + 1, wick_center_x] = candle_color

    # Convert numpy array to PIL Image
    img = Image.fromarray(image_data, 'L')
    return img

print("Candlestick image reconstruction function defined successfully.")

Candlestick image reconstruction function defined successfully.


# Task
Save the trained PyTorch model's state dictionary to a file named `candlestick_predictor_model.pth`.

## Save Model

### Subtask:
Save the trained PyTorch model's state dictionary to a file using `torch.save`.


**Reasoning**:
The subtask requires saving the trained PyTorch model's state dictionary to a file. This code block will use `torch.save()` to store the `model.state_dict()` to the specified filename.



In [21]:
torch.save(model.state_dict(), 'candlestick_predictor_model.pth')

print("Model state dictionary saved to 'candlestick_predictor_model.pth'")

Model state dictionary saved to 'candlestick_predictor_model.pth'


## Final Task

### Subtask:
Confirm that the model has been successfully saved and is ready for future loading and use.
